In [5]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import os
import cv2

In [6]:
prototxtPath=os.path.sep.join([r'C:\Users\Yashwanth Brahma\Documents\Mask-Detection\face-detectors','deploy.prototxt'])
weightsPath=os.path.sep.join([r'C:\Users\Yashwanth Brahma\Documents\Mask-Detection\face-detectors','res10_300x300_ssd_iter_140000.caffemodel'])

In [7]:
net=cv2.dnn.readNet(prototxtPath,weightsPath)

In [8]:
model=load_model(r'C:\Users\Yashwanth Brahma\Documents\Mask-Detection\mobilenet_v2.model')

In [93]:
image=cv2.imread(r'C:\Users\Yashwanth Brahma\Documents\Mask-Detection\ex1.jpg')

In [94]:
image

array([[[ 22,  34,  36],
        [ 12,  22,  22],
        [  7,  12,  13],
        ...,
        [ 26,  26,  26],
        [ 22,  22,  22],
        [ 65,  65,  65]],

       [[ 22,  34,  36],
        [ 12,  22,  22],
        [  7,  12,  13],
        ...,
        [ 23,  23,  23],
        [ 19,  19,  19],
        [ 63,  63,  63]],

       [[ 22,  34,  36],
        [ 12,  22,  22],
        [  7,  12,  13],
        ...,
        [ 19,  19,  19],
        [ 16,  16,  16],
        [ 60,  60,  60]],

       ...,

       [[124, 107,  98],
        [117, 104,  96],
        [100,  92,  92],
        ...,
        [253, 242, 215],
        [254, 243, 216],
        [254, 243, 216]],

       [[121, 106,  97],
        [116, 105,  97],
        [ 99,  91,  91],
        ...,
        [249, 238, 211],
        [250, 239, 212],
        [251, 240, 213]],

       [[121, 106,  97],
        [111, 100,  92],
        [101,  93,  93],
        ...,
        [251, 240, 213],
        [252, 241, 214],
        [253, 242, 215]]

In [95]:
(h,w)=image.shape[:2]
(h,w)

(182, 277)

In [96]:
blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))

In [97]:
blob

array([[[[ -82.,  -91.,  -96., ...,  -79.,  -77.,  -39.],
         [ -82.,  -91.,  -96., ...,  -80.,  -78.,  -40.],
         [ -82.,  -91.,  -96., ...,  -82.,  -80.,  -41.],
         ...,
         [  17.,   12.,   -2., ...,  145.,  146.,  147.],
         [  17.,   10.,   -2., ...,  146.,  147.,  148.],
         [  17.,    8.,   -1., ...,  147.,  148.,  149.]],

        [[-143., -154., -163., ..., -152., -150., -112.],
         [-143., -154., -163., ..., -153., -151., -113.],
         [-143., -154., -163., ..., -155., -153., -114.],
         ...,
         [ -71.,  -72.,  -83., ...,   61.,   62.,   63.],
         [ -71.,  -75.,  -83., ...,   62.,   63.,   64.],
         [ -71.,  -76.,  -83., ...,   63.,   64.,   65.]],

        [[ -87., -100., -108., ...,  -98.,  -96.,  -58.],
         [ -87., -100., -108., ...,  -99.,  -97.,  -59.],
         [ -87., -100., -109., ..., -101.,  -99.,  -60.],
         ...,
         [ -26.,  -26.,  -31., ...,   88.,   89.,   90.],
         [ -26.,  -29.,  -

In [98]:
net.setInput(blob)
detections=net.forward()

In [99]:
for i in range(0,detections.shape[2]):
    confidence=detections[0,0,i,2]
    
    if confidence>0.5:
            box=detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startx,starty,endx,endy)=box.astype('int')
            (startx,starty)=(max(0,startx),max(0,starty))
            (endx,endy)=(min(w-1,endx),min(h-1,endy))
            
            face=image[starty:endy, startx:endx]
            face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
            face=cv2.resize(face,(224,224))
            face=img_to_array(face)
            face=preprocess_input(face)
            face=np.expand_dims(face,axis=0)
            
            (mask,withoutMask)=model.predict(face)[0]
            
            label='Mask' if mask>withoutMask else 'No Mask'
            color=(0,255,0) if label=='Mask' else (0,0,255)
            
            label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
            
            cv2.putText(image,label,(startx,starty-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
            cv2.rectangle(image,(startx,starty),(endx,endy),color,2)
            

cv2.imshow("outPut",image)
cv2.waitKey(0)
cv2.destroyAllWindows()